```{contents}
```
## Reflection Loop 

A **Reflection Loop** in LangGraph is a **cyclic execution pattern** where an LLM (or group of agents) repeatedly **generates an output, critiques it, and refines it**, until a quality criterion is satisfied.
This enables **self-improvement, error correction, reasoning stabilization, and higher reliability**.

---

### **1. Core Intuition**

Humans solve problems by:

> Draft → Review → Fix → Repeat

LangGraph formalizes this cognitive process as a **controlled feedback cycle** in the execution graph.

---

### **2. Conceptual Structure**

```
        ┌─────────┐
        │ Draft   │
        └────┬────┘
             ↓
        ┌─────────┐
        │ Reflect │   ← Critique quality, detect flaws
        └────┬────┘
             ↓
        ┌─────────┐
        │ Revise  │   ← Improve based on critique
        └────┬────┘
             ↓
           (Loop until satisfied)
```

Termination occurs when the reflection step signals **acceptable quality**.

---

### **3. Why Reflection Is Necessary for LLM Systems**

| Problem Without Reflection | Benefit With Reflection |
| -------------------------- | ----------------------- |
| Hallucinations             | Self-correction         |
| Shallow reasoning          | Deeper analysis         |
| Logical errors             | Consistency             |
| Incoherent output          | Structured answers      |
| Single-pass failure        | Iterative improvement   |

---

### **4. State Design for Reflection**

```python
class State(TypedDict):
    draft: str
    critique: str
    final: str
    quality_ok: bool
```

---

### **5. Minimal LangGraph Implementation**

```python
from langgraph.graph import StateGraph, END
from typing import TypedDict

class State(TypedDict):
    draft: str
    critique: str
    quality_ok: bool

def generate(state):
    return {"draft": "Initial answer with possible mistakes"}

def reflect(state):
    critique = "The answer lacks clarity."
    ok = "lacks" not in critique
    return {"critique": critique, "quality_ok": ok}

def revise(state):
    improved = state["draft"] + " (improved)"
    return {"draft": improved}

builder = StateGraph(State)

builder.add_node("generate", generate)
builder.add_node("reflect", reflect)
builder.add_node("revise", revise)

builder.set_entry_point("generate")
builder.add_edge("generate", "reflect")
builder.add_conditional_edges(
    "reflect",
    lambda s: END if s["quality_ok"] else "revise",
    {"revise": "revise", END: END}
)
builder.add_edge("revise", "reflect")

graph = builder.compile()
result = graph.invoke({})
```

---

### **6. Execution Trace**

```
Generate → Reflect (fail) → Revise → Reflect (fail) → Revise → Reflect (pass) → End
```

---

### **7. Production Enhancements**

| Mechanism           | Purpose                       |
| ------------------- | ----------------------------- |
| Max iterations      | Prevent infinite loops        |
| Human approval gate | High-risk tasks               |
| Scoring model       | Objective quality measurement |
| Checkpointing       | Safe recovery                 |
| Cost tracking       | Budget control                |

```python
graph.invoke(input, config={"recursion_limit": 10})
```

---

### **8. Common Variants**

| Variant           | Use Case                         |
| ----------------- | -------------------------------- |
| Self-Reflection   | Single agent improves own output |
| Peer Review       | One agent critiques another      |
| Debate Reflection | Multiple critics refine output   |
| Automated QA      | Code and document verification   |
| Tool-Assisted     | Use analyzers, linters, tests    |

---

### **9. Where Reflection Loops Are Used**

* Autonomous agents
* Code generation & debugging
* Research synthesis
* Planning systems
* Safety-critical workflows
* Decision support systems

---

### **10. Mental Model**

LangGraph reflection loops implement:

> **Controlled cognitive feedback**

which transforms LLMs from **one-shot text generators** into **self-correcting reasoning systems**.


### Demonstration

In [1]:
from langgraph.graph import StateGraph, END
from typing import TypedDict

# -----------------------------
# 1. Define Shared State
# -----------------------------
class State(TypedDict):
    draft: str
    critique: str
    quality_ok: bool

# -----------------------------
# 2. Define Nodes
# -----------------------------
def generate(state: State) -> State:
    print("\n--- GENERATE ---")
    return {"draft": "The capital of France is Berlin."}

def reflect(state: State) -> State:
    print("\n--- REFLECT ---")
    critique = "Incorrect capital." if "Berlin" in state["draft"] else "Looks good."
    return {
        "critique": critique,
        "quality_ok": critique == "Looks good."
    }

def revise(state: State) -> State:
    print("\n--- REVISE ---")
    fixed = state["draft"].replace("Berlin", "Paris")
    return {"draft": fixed}

# -----------------------------
# 3. Build Graph
# -----------------------------
builder = StateGraph(State)

builder.add_node("generate", generate)
builder.add_node("reflect", reflect)
builder.add_node("revise", revise)

builder.set_entry_point("generate")
builder.add_edge("generate", "reflect")

builder.add_conditional_edges(
    "reflect",
    lambda s: END if s["quality_ok"] else "revise",
    {"revise": "revise", END: END}
)

builder.add_edge("revise", "reflect")

# -----------------------------
# 4. Compile & Run
# -----------------------------
graph = builder.compile()

result = graph.invoke({}, config={"recursion_limit": 5})

print("\nFINAL RESULT:")
print(result)



--- GENERATE ---

--- REFLECT ---

--- REVISE ---

--- REFLECT ---

FINAL RESULT:
{'draft': 'The capital of France is Paris.', 'critique': 'Looks good.', 'quality_ok': True}
